# Prueba de Proveedores de IA

Este notebook permite probar diferentes proveedores de IA y cambiar entre ellos dinámicamente.

In [ ]:
import os
import sys
import json
from dotenv import load_dotenv
from pathlib import Path

# Verificar que estamos en el directorio correcto o ajustar el path
project_dir = Path.cwd()
if not (project_dir / "providers").exists():
    # Si no estamos en el directorio raíz, intentar encontrarlo
    if (project_dir.parent / "providers").exists():
        project_dir = project_dir.parent
    else:
        # Otro intento, buscar hacia arriba en el árbol de directorios
        for parent in project_dir.parents:
            if (parent / "providers").exists():
                project_dir = parent
                break

# Añadir el directorio raíz al path de Python
sys.path.insert(0, str(project_dir))

# Cargar variables de entorno
load_dotenv(project_dir / ".env")

# Verificar API keys disponibles
api_keys = {
    "OpenAI": os.getenv("OPENAI_API_KEY"),
    "Claude": os.getenv("CLAUDE_API_KEY") or os.getenv("ANTHROPIC_API_KEY"),
    "Otros": [key for key in os.environ.keys() if "API_KEY" in key and key not in ["OPENAI_API_KEY", "CLAUDE_API_KEY", "ANTHROPIC_API_KEY"]]
}

print(f"📂 Directorio del proyecto: {project_dir}")
print("🔑 API Keys disponibles:")
for provider, key in api_keys.items():
    if provider != "Otros":
        print(f"  - {provider}: {'✓ Disponible' if key else '❌ No disponible'}")
    elif key:
        print(f"  - Otras claves: {', '.join(key)}")

## Importar y configurar proveedores

Importamos los componentes necesarios de nuestro sistema de proveedores.

In [ ]:
# Importar los módulos necesarios
try:
    from providers.base_provider import BaseProvider
    from providers.openai_provider import OpenAIProvider
    from providers.claude_provider import ClaudeProvider
    from providers.fake_provider import FakeProvider
    from providers.provider_manager import ProviderManager
    print("✅ Módulos importados correctamente")
except ImportError as e:
    print(f"❌ Error importando módulos: {e}")
    print("Asegúrate de que estás ejecutando este notebook desde el directorio correcto")
    print("o que has ajustado sys.path adecuadamente.")

## Crear el archivo de configuración temporal

Si no existe un archivo de configuración, creemos uno temporal.

In [ ]:
import yaml

# Ruta al archivo de configuración
config_dir = project_dir / "config"
config_file = config_dir / "settings.yaml"

# Crear directorio de configuración si no existe
config_dir.mkdir(exist_ok=True)

# Configuración predeterminada
default_config = {
    "debug": True,
    "log_level": "INFO",
    "default_provider": "openai" if api_keys["OpenAI"] else ("claude" if api_keys["Claude"] else "fake"),
    "providers": {
        "openai": {
            "enabled": bool(api_keys["OpenAI"]),
            "model": "gpt-4o",
            "max_tokens": 500,
            "temperature": 0.7
        },
        "claude": {
            "enabled": bool(api_keys["Claude"]),
            "model": "claude-3-5-sonnet-20240620",
            "max_tokens": 500,
            "temperature": 0.7
        },
        "fake": {
            "enabled": True,
            "mode": "realistic",
            "delay": 0.5
        }
    }
}

# Si el archivo no existe, crearlo
if not config_file.exists():
    with open(config_file, 'w') as f:
        yaml.dump(default_config, f, default_flow_style=False)
    print(f"✅ Archivo de configuración creado: {config_file}")
else:
    print(f"ℹ️ Usando archivo de configuración existente: {config_file}")
    
    # Leer la configuración actual
    with open(config_file, 'r') as f:
        current_config = yaml.safe_load(f)
    
    # Validar y actualizar la configuración si es necesario
    if "providers" not in current_config:
        print("⚠️ El archivo de configuración no tiene una sección 'providers'. Actualizando...")
        current_config["providers"] = default_config["providers"]
        with open(config_file, 'w') as f:
            yaml.dump(current_config, f, default_flow_style=False)

## Inicializar el Gestor de Proveedores

Ahora inicializamos el gestor de proveedores con la configuración.

In [ ]:
# Inicializar el gestor de proveedores
manager = None
try:
    manager = ProviderManager(config_file)
    print(f"✅ Gestor de proveedores inicializado")
    
    # Mostrar proveedores disponibles
    available_providers = manager.list_available_providers()
    current_provider = manager.get_current_provider_name()
    
    print(f"📊 Proveedores disponibles: {', '.join(available_providers)}")
    print(f"🔍 Proveedor actual: {current_provider}")
except Exception as e:
    print(f"❌ Error inicializando el gestor de proveedores: {e}")

## Función de Utilidad para Cambiar Proveedores

Creamos una función para facilitar el cambio entre proveedores.

In [ ]:
def switch_provider(provider_name=None):
    """Cambia al proveedor especificado o muestra los disponibles si no se especifica ninguno."""
    if not manager:
        print("❌ El gestor de proveedores no está inicializado")
        return None
    
    if not provider_name:
        available = manager.list_available_providers()
        current = manager.get_current_provider_name()
        print(f"📊 Proveedores disponibles: {', '.join(available)}")
        print(f"🔍 Proveedor actual: {current}")
        return manager.get_provider()
    
    try:
        provider = manager.set_provider(provider_name)
        print(f"✅ Cambiado a proveedor: {provider_name}")
        return provider
    except Exception as e:
        print(f"❌ Error al cambiar al proveedor '{provider_name}': {e}")
        return None

# Probar la función
current_provider = switch_provider()

## Función para Generar Texto

Vamos a crear una función para generar texto con el proveedor actual o con uno específico.

In [ ]:
def generate_text(prompt, provider_name=None, **kwargs):
    """Genera texto usando el proveedor actual o el especificado."""
    if not manager:
        print("❌ El gestor de proveedores no está inicializado")
        return None
    
    try:
        # Obtener el proveedor adecuado
        provider = manager.get_provider(provider_name)
        provider_name = provider_name or manager.get_current_provider_name()
        
        print(f"🤖 Generando texto con proveedor: {provider_name}")
        print(f"📝 Prompt: {prompt[:50]}..." if len(prompt) > 50 else f"📝 Prompt: {prompt}")
        
        # Parámetros predeterminados
        params = {
            "max_tokens": 200,
            "temperature": 0.7
        }
        # Actualizar con parámetros proporcionados
        params.update(kwargs)
        
        # Llamar al proveedor
        import time
        start_time = time.time()
        response = provider.generate_text(prompt, **params)
        elapsed_time = time.time() - start_time
        
        print(f"⏱️ Tiempo de respuesta: {elapsed_time:.2f} segundos")
        return response
    except Exception as e:
        print(f"❌ Error generando texto: {e}")
        return None

## Función para Generar Tareas

También creamos una función para generar tareas estructuradas (JSON).

In [ ]:
def generate_tasks(objective, context=None, provider_name=None):
    """Genera tareas estructuradas para un objetivo."""
    if not manager:
        print("❌ El gestor de proveedores no está inicializado")
        return None
    
    try:
        # Obtener el proveedor adecuado
        provider = manager.get_provider(provider_name)
        provider_name = provider_name or manager.get_current_provider_name()
        
        print(f"🤖 Generando tareas con proveedor: {provider_name}")
        print(f"🎯 Objetivo: {objective}")
        if context:
            print(f"📋 Contexto proporcionado: {len(str(context))} caracteres")
        
        # Llamar al proveedor
        import time
        start_time = time.time()
        tasks = provider.generar_tareas(objective, context)
        elapsed_time = time.time() - start_time
        
        print(f"⏱️ Tiempo de respuesta: {elapsed_time:.2f} segundos")
        print(f"📋 Se generaron {len(tasks)} tareas")
        
        return tasks
    except Exception as e:
        print(f"❌ Error generando tareas: {e}")
        return None

## Prueba Rápida

Hagamos una prueba rápida con algunos proveedores.

In [ ]:
# Probar OpenAI (si está disponible)
if "openai" in manager.list_available_providers():
    switch_provider("openai")
    response = generate_text("Hola, ¿cómo estás?")
    print(f"\nRespuesta:\n{response}\n" + "-"*50)

In [ ]:
# Probar Claude (si está disponible)
if "claude" in manager.list_available_providers():
    switch_provider("claude")
    response = generate_text("Hola, ¿cómo estás?")
    print(f"\nRespuesta:\n{response}\n" + "-"*50)

In [ ]:
# Probar el proveedor de respaldo (fake)
if "fake" in manager.list_available_providers():
    switch_provider("fake")
    response = generate_text("Hola, ¿cómo estás?")
    print(f"\nRespuesta:\n{response}\n" + "-"*50)

## Prueba de Generación de Tareas

Ahora probemos la generación de tareas estructuradas con diferentes proveedores.

In [ ]:
# Probar generación de tareas con OpenAI
if "openai" in manager.list_available_providers():
    switch_provider("openai")
    tasks = generate_tasks("Crear un sistema de monitoreo de servidores")
    
    if tasks:
        print("\nTareas generadas:")
        for i, task in enumerate(tasks, 1):
            print(f"  {i}. {task.get('tarea', task.get('descripcion', 'Sin descripción'))}")
        print("-"*50)

In [ ]:
# Probar generación de tareas con Claude
if "claude" in manager.list_available_providers():
    switch_provider("claude")
    tasks = generate_tasks("Crear un sistema de monitoreo de servidores")
    
    if tasks:
        print("\nTareas generadas:")
        for i, task in enumerate(tasks, 1):
            print(f"  {i}. {task.get('tarea', task.get('descripcion', 'Sin descripción'))}")
        print("-"*50)

## Función para Generar Respuestas en JSON

Creamos una función para obtener respuestas estructuradas en formato JSON.

In [ ]:
import json
import re
import datetime

def generate_structured_response(prompt, schema=None, provider_name=None):
    """Genera una respuesta estructurada en formato JSON."""
    if not manager:
        print("❌ El gestor de proveedores no está inicializado")
        return None
    
    # Esquema predeterminado si no se proporciona
    if not schema:
        schema = {
            "type": "object",
            "properties": {
                "items": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "id": {"type": "integer"},
                            "titulo": {"type": "string"},
                            "descripcion": {"type": "string"}
                        }
                    }
                },
                "metadata": {
                    "type": "object",
                    "properties": {
                        "total": {"type": "integer"},
                        "generado": {"type": "string", "format": "date-time"}
                    }
                }
            }
        }
    
    schema_str = json.dumps(schema, indent=2)
    
    # Enriquecer el prompt
    enriched_prompt = f"""
    {prompt}
    
    Por favor, proporciona tu respuesta en formato JSON válido según el siguiente esquema:
    {schema_str}
    
    Responde ÚNICAMENTE con JSON válido.
    """
    
    try:
        # Obtener respuesta del proveedor
        provider = manager.get_provider(provider_name)
        provider_name = provider_name or manager.get_current_provider_name()
        
        print(f"🤖 Generando respuesta JSON con proveedor: {provider_name}")
        
        # Generar respuesta
        response = provider.generate_text(
            prompt=enriched_prompt,
            temperature=0.3  # Temperatura baja para respuestas más deterministas
        )
        
        # Extraer JSON
        json_pattern = r'```(?:json)?(.*?)```'
        json_match = re.search(json_pattern, response, re.DOTALL)
        
        if json_match:
            json_str = json_match.group(1).strip()
        else:
            # Si no está en backticks, intentar procesar directamente
            json_str = response.strip()
        
        # Intentar parsear el JSON
        result = json.loads(json_str)
        
        print(f"✅ JSON generado y parseado correctamente")
        return result
    except json.JSONDecodeError as e:
        print(f"❌ Error parseando JSON: {e}")
        print(f"Respuesta original:\n{response}")
        return None
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

## Probando la generación de respuestas JSON

Ahora probemos la generación de respuestas estructuradas.

In [ ]:
# Probar generación de JSON con OpenAI
if "openai" in manager.list_available_providers():
    switch_provider("openai")
    result = generate_structured_response("Genera 3 ideas de nombres para una empresa de tecnología")
    
    if result:
        print("\nResultado:\n")
        print(json.dumps(result, indent=2))
        print("-"*50)

In [ ]:
# Probar generación de JSON con Claude
if "claude" in manager.list_available_providers():
    switch_provider("claude")
    result = generate_structured_response("Genera 3 ideas de nombres para una empresa de tecnología")
    
    if result:
        print("\nResultado:\n")
        print(json.dumps(result, indent=2))
        print("-"*50)

## Generación y Comparación

Finalmente, comparemos las respuestas de diferentes proveedores para el mismo prompt.

In [ ]:
def compare_providers(prompt, providers=None):
    """Compara respuestas de diferentes proveedores para el mismo prompt."""
    if not manager:
        print("❌ El gestor de proveedores no está inicializado")
        return
    
    # Si no se especifican proveedores, usar todos los disponibles
    if not providers:
        providers = manager.list_available_providers()
    
    results = {}
    times = {}
    
    print(f"🔄 Comparando proveedores para prompt: '{prompt}'\n")
    
    for provider_name in providers:
        print(f"📊 Probando proveedor: {provider_name}")
        try:
            import time
            start_time = time.time()
            
            provider = manager.get_provider(provider_name)
            response = provider.generate_text(prompt, max_tokens=100)
            
            elapsed_time = time.time() - start_time
            results[provider_name] = response
            times[provider_name] = elapsed_time
            
            print(f"⏱️ Tiempo: {elapsed_time:.2f} segundos")
            print(f"📝 Longitud: {len(response)} caracteres\n")
        except Exception as e:
            print(f"❌ Error con proveedor {provider_name}: {e}\n")
    
    print("\n📈 Resultados:\n" + "-"*50)
    for provider_name, response in results.items():
        print(f"\n🤖 {provider_name} ({times[provider_name]:.2f}s):\n{response}\n" + "-"*50)
    
    return results

# Comparar proveedores
available = manager.list_available_providers()
if len(available) > 1:
    compare_providers("Explica qué es la programación funcional en una frase corta.")